In [ ]:
# Only enable critical logging (Optional)
import os
os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "CRITICAL"

In [ ]:
def what_is_installed():
    from pycaret import show_versions
    show_versions()

try:
    what_is_installed()
except ModuleNotFoundError:
    !pip install pycaret
    what_is_installed()

In [1]:
import time
import numpy as np
import pandas as pd

from pycaret.datasets import get_data
from pycaret.time_series import TSForecastingExperiment

ModuleNotFoundError: No module named 'pycaret'

In [ ]:
#Import All Library
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
!pip install timeseries-cv
import tsxv
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from keras.callbacks import EarlyStopping

In [ ]:
# load time series dataset
#df = read_csv('/kaggle/input/data-emas-skripsi/gold prices full.csv', header=0, index_col=1).drop("Unnamed: 0", axis=1)
df = pd.read_csv('/kaggle/input/data-emas-skripsi/gold prices.csv',index_col='index',parse_dates=True).drop("Unnamed: 0", axis=1)
df.head()
df.tail()
df.dtypes

**CARA LAIN**

In [ ]:
from sklearn.preprocessing import StandardScaler

requests = df
#Scale the data
print("Request Range before scaling : " , 
          min(requests.values),
          max(requests.values))

scaler = StandardScaler()
scaled_requests=scaler.fit_transform(requests)
print("Request Range after scaling : " , 
          min(scaled_requests),
          max(scaled_requests))

#Training data has to be sequencial - first 4 weeks
values = df.values
train_size = math.ceil(len(values)* 0.6)

#Number of samples to lookback for each sample
lookback= math.ceil(len(values)* 0.6)-(365*5)

#Separate training and test data
train_requests = scaled_requests[0:train_size,:]

#Add an additional week for lookback.
test_requests = scaled_requests[train_size-lookback:,:]

print("\n Shaped of Train, Test : ",
      train_requests.shape, test_requests.shape)

In [ ]:
#Each data point (X) is linked to the previous data points of size=lookback
#The predicted value (Y) is the next point

def create_rnn_dataset(data, lookback=1):
    
    data_x, data_y = [], []
    for i in range(len(data)- lookback -1):
            #All points from this point, looking backwards upto lookback
            a = data[i:(i+ lookback), 0]
            data_x.append(a)
            #The next point
            data_y.append(data[i + lookback, 0])
    return np.array(data_x), np.array(data_y)

#Create X and Y for training
train_req_x, train_req_y = create_rnn_dataset(train_requests,lookback)

#Reshape for use with LSTM
train_req_x = np.reshape(train_req_x, 
                         (train_req_x.shape[0],1, train_req_x.shape[1]))

print("Shapes of X, Y: ",train_req_x.shape, train_req_y.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense
import tensorflow as tf

tf.random.set_seed(211)

#Create a Keras Model
ts_model=Sequential()
#Add LSTM
ts_model.add(LSTM(50, return_sequences=True, input_shape=(1,lookback)))
ts_model.add(LSTM(50, return_sequences=False))
ts_model.add(Dense(1))

opt = keras.optimizers.Adam(learning_rate=0.01)
es = EarlyStopping(monitor='loss', patience = 7)

#Compile with Adam Optimizer. Optimize for minimum mean square error
ts_model.compile(loss="mean_squared_error",
                 optimizer=opt)

#Print model summary
ts_model.summary()

#Train the model
ts_model.fit(train_req_x, train_req_y, 
             epochs=100, batch_size=56, verbose=1, callbacks=[es])

In [ ]:
#Preprocess the test dataset, the same way training set is processed
test_req_x, test_req_y = create_rnn_dataset(test_requests,lookback)
test_req_x = np.reshape(test_req_x, 
                         (test_req_x.shape[0],1, test_req_x.shape[1]))

#Evaluate the model
ts_model.evaluate(test_req_x, test_req_y, verbose=1)

#Predict for the training dataset
predict_on_train= ts_model.predict(train_req_x)
#Predict on the test dataset
predict_on_test = ts_model.predict(test_req_x)

#Inverse the scaling to view results
predict_on_train = scaler.inverse_transform(predict_on_train)
predict_on_test = scaler.inverse_transform(predict_on_test)

In [ ]:
#Use last part of the training data as the initial lookback
curr_input= test_req_x[-1,:].flatten()

#Predict for the next week
predict_for = math.ceil(len(values)* 0.4)#(366+365)+(365*2)
#predict_for = (366+365)+(365*2)

for i in range(predict_for):
    
    #Take the last lookback no. of samples as X
    this_input=curr_input[-lookback:]
    #Create the input
    this_input=this_input.reshape((1,1,lookback))
    #Predict for the next point
    this_prediction=ts_model.predict(this_input)

    #Add the current prediction to the input
    curr_input = np.append(curr_input,this_prediction.flatten())

#Extract the last predict_for part of curr_input, which contains all the new predictions
predict_on_future=np.reshape(np.array(curr_input[-predict_for:]),(predict_for,1))

# #Inverse to view results
predict_on_future=scaler.inverse_transform(predict_on_future)

print(predict_on_future[:5])

In [ ]:
df_tanggal = pd.DataFrame({'index':pd.date_range("2024-01-01", periods=len(predict_on_future))})
future_pred = pd.DataFrame(predict_on_future, index = df_tanggal['index'])
future_pred.tail()

In [ ]:
future_pred.set_axis(["value"], axis="columns", inplace=True)
future_pred.head(5)

In [ ]:
future_pred.loc[pd.Timestamp('2023-12-31')] = 2078.4
df_new = future_pred.sort_index().asfreq('D', method='pad')
df_new.head()

In [ ]:
#Labels and titles
#plt.figure(figsize=(17, 10))
plt.xlabel("Year")
plt.ylabel("Gold Price (USD/troy ounce)")
#plt.title("Gold Price Forecasting")

#First fold- CV
plt.plot(df, label='Actual Values', color="black")
plt.plot(df_new.index, df_new, label='Forecast', color="red")
plt.legend(loc='best')

In [ ]:
#plt.figure(figsize=(10, 5))
plt.plot(df.loc['2020-01-01':'2023-12-31'], color="black", label="Actual Values")
plt.plot(df_new.index, df_new, color="RED", label="Forecast")
plt.xlabel("Year")
plt.ylabel("Gold Price (USD/troy ounce)")
plt.legend(loc='best')
plt.show()

In [ ]:
df_new.to_excel('forecast_best.xlsx')